In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/original-data/data.json


In [2]:
!pip install wikipedia
!pip install rank_bm25
import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... - done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=51942cd48c3b6933352b02ab0a40d3e9507ca23092d62049e5086238f2aedcaa
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import json

# Import from JSON file
with open('/kaggle/input/original-data/data.json', 'r') as json_file:
    data = json.load(json_file)

# Now, imported_data is a list of dictionaries


In [4]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
content = [page['content'][:10000] for page in data]

In [6]:
from tqdm import tqdm
i = 0
final = {}
for flag in tqdm(content):
  sentences = nltk.tokenize.sent_tokenize(flag)
  length = 0
  chunk = ""
  chunks = []
  count = -1

  for sentence in sentences:
      count += 1
      combined_length = len(tokenizer.tokenize(sentence)) + length

      if combined_length <= tokenizer.max_len_single_sentence:
          chunk += sentence + " "
          length = combined_length

          if count == len(sentences) - 1:
              chunks.append(chunk.strip())  # save the chunk
      else:
          # Handle the case where a sentence itself exceeds the maximum length
          if len(tokenizer.tokenize(sentence)) > tokenizer.max_len_single_sentence:
              # Split the sentence into chunks that fit the maximum length
              sub_chunks = [sentence[i:i + tokenizer.max_len_single_sentence]
                            for i in range(0, len(sentence), tokenizer.max_len_single_sentence)]
              chunks.extend(sub_chunks)
          else:
              chunks.append(chunk.strip())  # save the chunk
              length = 0
              chunk = ""
              chunk += sentence + " "
              length = len(tokenizer.tokenize(sentence))
  inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
  c_1 = ""
  for input in inputs:
    output = model.generate(**input)
    x = tokenizer.decode(output[0], skip_special_tokens=True)
    print(x)
    c_1 += x + " "
  final[i] = c_1
  i += 1

  0%|          | 0/100 [00:00<?, ?it/s]

Solar energy is radiant light and heat from the Sun that is harnessed using a range of technologies.<n>Active solar techniques include the use of photovoltaic systems, concentrated solar power, and solar water heating.<n>The total solar energy absorbed by Earth's atmosphere, oceans and land masses is approximately 122 PWyear = 3,850,000 exajoules (EJ) per year.


  1%|          | 1/100 [01:09<1:55:00, 69.71s/it]

Solar hot water systems use sunlight to heat water.<n>China is the world leader in their deployment with 309 GWth installed.<n>Israel and Cyprus are the per capita leaders in the use of solar hot water systems.
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia.<n>Washington led Patriot forces to victory in the American Revolutionary War.<n>He was twice elected president by the Electoral College unanimously.


  2%|▏         | 2/100 [02:10<1:44:58, 64.27s/it]

...

8% of Internet users are very active in content creation and consumption.<n>69 percent of American and European internet users are "spectators," who consume—but don't create—online and digital media.<n> Motivations for creating new content may include the desire to gain new knowledge, the possibility of publicity, or simple altruism.
Digital marketing began in 1990 when the Archie search engine was created as an index for FTP sites.<n>In the 2000s, customers began searching for products and making decisions about their needs online first.<n>With the development of social media in the 2000s, such as LinkedIn, Facebook, YouTube and Twitter, consumers became highly dependent on digital electronics in daily lives.


 99%|█████████▉| 99/100 [1:48:50<01:03, 63.37s/it]

Businesses are focusing more on digital marketing efforts on cultivating brand recognition and recall.<n>Increasing brand awareness is a focus of digital marketing strategy for a number of reasons.<n>One in three U.S. consumers are influenced by social media when buying a product, while 47% of millennials factor their interaction with a brand on social when making a purchase.


100%|██████████| 100/100 [1:49:33<00:00, 65.73s/it]

The United States of America is a federal republic consisting of 50 states, a federal district, five major territories, and various minor islands.<n>Each state has its own constitution and government, and all states and their residents are represented in the federal Congress.<n>With the exception of Navassa Island, Puerto Rico, and the U.S. Virgin Islands, all territories are located in the Pacific Ocean.


In [7]:
import json

In [8]:
output_file_path = 'pegasus.txt'

# Convert the dictionary to JSON format
json_data = json.dumps(final, indent=2)

# Write the JSON data to a text file
with open(output_file_path, 'w') as file:
    file.write(json_data)

print(f'Dictionary has been exported to {output_file_path}')

Dictionary has been exported to pegasus.txt
